In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark import Session
from snowflake.snowpark.types import StringType

session = get_active_session()


In [ ]:
from snowflake.snowpark import Session, DataFrame

def process_data(session: Session, month_date: str, event_date: str, load_date: str):
    delete_sql = f""" DELETE FROM TELKOMSEL_POC.CURATED.merge_revenue_ifrs_dd_upcc WHERE event_date = '{event_date}' """

    session.sql(delete_sql).collect()

    # Step 2: Build the DataFrame with new data to append
    query = f"""
     SELECT trx_date,
           concat(substr(ts_ifrs,1,4), '-', substr(ts_ifrs,5,2), '-', substr(ts_ifrs,7,2)) AS purchase_date,
           quota_extended_attribute AS transaction_id,
           subs_id,
           msisdn,
           price_plan_id,
           brand,
           pre_post_flag,
           cust_type_desc,
           cust_subtype_desc,
           customer_sub_segment,
           lac,
           ci,
           lacci_id_ifrs AS lacci_id,
           node_type AS node,
           CASE WHEN area_sales_ifrs IS NULL OR area_sales_ifrs='' THEN 'UNKNOWN' ELSE area_sales_ifrs END AS area_sales,
           CASE WHEN region_sales_ifrs IS NULL OR region_sales_ifrs='' THEN 'UNKNOWN' ELSE region_sales_ifrs END AS region_sales,
           CASE WHEN branch_ifrs IS NULL OR branch_ifrs='' THEN 'UNKNOWN' ELSE branch_ifrs END AS branch,
           CASE WHEN sub_branch_ifrs IS NULL OR sub_branch_ifrs='' THEN 'UNKNOWN' ELSE sub_branch_ifrs END AS subbranch,
           CASE WHEN cluster_ifrs IS NULL OR cluster_ifrs='' THEN 'UNKNOWN' ELSE cluster_ifrs END AS cluster_sales,
           CASE WHEN provinsi_ifrs IS NULL OR provinsi_ifrs='' THEN 'UNKNOWN' ELSE provinsi_ifrs END AS provinsi,
           CASE WHEN kabupaten_ifrs IS NULL OR kabupaten_ifrs='' THEN 'UNKNOWN' ELSE kabupaten_ifrs END AS kabupaten,
           CASE WHEN kecamatan_ifrs IS NULL OR kecamatan_ifrs='' THEN 'UNKNOWN' ELSE kecamatan_ifrs END AS kecamatan,
           CASE WHEN kelurahan_ifrs IS NULL OR kelurahan_ifrs='' THEN 'UNKNOWN' ELSE kelurahan_ifrs END AS kelurahan,
           lacci_closing_flag_ifrs AS lacci_closing_flag,
           sigma_business_id,
           sigma_rules_id,
           package_id AS sku,
           l1_payu,
           l2_service_type,
           l3_allowance_type,
           l4_product_category,
           l5_product,
           '' AS l1_ias,
           '' AS l2_ias,
           '' AS l3_ias,
           commercial_name,
           channel,
           validity AS pack_validity,
           cast(sum(rev_per_usage) AS decimal (38,15)) AS rev_per_usage,
           cast(sum(rev_seized) AS decimal (38,15)) AS rev_seized,
           cast(sum(0) AS int) AS dur,
           cast(count(*) AS int) AS trx,
           cast(sum(quota_usage) AS bigint) AS vol,
           cust_id,
           service_name AS profile_name,
           quota_name,
           COALESCE(split(ifrs_param,'[\$]')[9],'0') AS service_filter,
           '' AS price_plan_name,
           channel_id,
           site_id,
           site_name,
           region_hlr,
           city_hlr,
           '{load_date}'::varchar AS load_date,
           event_date,
           CASE WHEN rev_seized > 0 THEN 'UPCC-SEIZED' ELSE 'UPCC_USAGE' END AS SOURCE

    FROM TELKOMSEL_POC.RAW.USAGE_UPCC_EDR_POC_TOKENIZED
    WHERE event_date = '{event_date}'::varchar
      AND substr(quota_extended_attribute, 19, 2) = '00'
      AND TRIGGER_TYPE in (2,3,18,100,136,138,111,119,116)
      AND pre_post_flag = '1'

    GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,45,46,47,48,49,50,51,52,53,54,55,56,57 
   
    """

    df = session.sql(query)

    # Step 3: Append the query result to the target table
    df.write.mode('append').save_as_table('TELKOMSEL_POC.CURATED.merge_revenue_ifrs_dd_upcc')

    return f"Deleted and appended data for event_date: {event_date}"


In [ ]:
session.sproc.register(
    func=process_data,
    name="process_data_merge_revenue_ifrs_dd_upcc",
    is_permanent=True,
    stage_location="@TSELPOC_STAGE",           # Replace with your actual stage
    replace=True,
    input_types=[StringType(), StringType()],          # Instead of [str]
    return_type=StringType(),            # Instead of str or "STRING"
    packages=["snowflake-snowpark-python"]
)


In [ ]:
call process_data_merge_revenue_ifrs_dd_upcc('2025-04-01','2025-04-02')